<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Higher_Dimensional_Cognition_in_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf

class HigherDimensionalAI(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(128, activation='relu')
        self.dense2 = tf.keras.layers.Dense(256, activation='relu')
        self.dense3 = tf.keras.layers.Dense(512, activation='relu')  # Higher-dim projection
        self.output_layer = tf.keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return self.output_layer(x)

# Reproducibility (optional, helpful in notebooks)
np.random.seed(42)
tf.random.set_seed(42)

# Generate training data
x_train = np.random.rand(1000, 10).astype(np.float32)
y_train = np.random.randint(0, 10, size=(1000,)).astype(np.int32)

# Train higher-dimensional AI
model = HigherDimensionalAI()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

def build_model(input_dim=10, num_classes=10, l2=1e-4, dropout=0.2):
    he = tf.keras.initializers.HeNormal()
    reg = tf.keras.regularizers.l2(l2)

    inputs = tf.keras.Input(shape=(input_dim,), name="features")
    x = tf.keras.layers.Dense(128, activation='relu', kernel_initializer=he, kernel_regularizer=reg)(inputs)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(256, activation='relu', kernel_initializer=he, kernel_regularizer=reg)(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Dense(512, activation='relu', kernel_initializer=he, kernel_regularizer=reg)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs, outputs, name="HigherDimensionalAI")

def main():
    # Reproducibility
    np.random.seed(42)
    tf.random.set_seed(42)

    # Data
    x = np.random.rand(1000, 10).astype(np.float32)
    y = np.random.randint(0, 10, size=(1000,)).astype(np.int32)

    # Model
    model = build_model(input_dim=10, num_classes=10, l2=1e-4, dropout=0.2)
    model.summary()

    # Callbacks
    cbs = [
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
    ]

    # Compile + train
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    model.fit(x, y, epochs=30, batch_size=64, validation_split=0.2, callbacks=cbs, verbose=1)

    # Evaluate
    loss, acc = model.evaluate(x, y, verbose=0)
    print(f"Final metrics — loss: {loss:.4f}, acc: {acc:.4f}")

if __name__ == "__main__":
    main()

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import argparse
import json
import os
import sys
from dataclasses import asdict, dataclass
from datetime import datetime
from typing import Any, Dict, List, Optional, Sequence, Tuple

import numpy as np
import tensorflow as tf


# ------------------------------
# Utilities
# ------------------------------

def log(msg: str) -> None:
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts}] {msg}", flush=True)


def ensure_dir(path: str) -> None:
    os.makedirs(path, exist_ok=True)


def save_json(path: str, data: Dict[str, Any]) -> None:
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, sort_keys=True)


def load_json(path: str) -> Dict[str, Any]:
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def sanitize_argv(argv: Sequence[str]) -> List[str]:
    """Remove Jupyter/Colab stray args like -f and *.json kernel files."""
    bad_flags = {"-f", "--f"}
    cleaned: List[str] = []
    skip_next = False
    for i, a in enumerate(argv):
        if skip_next:
            skip_next = False
            continue
        if a in bad_flags:
            # Skip this flag and its possible value
            if i + 1 < len(argv) and not argv[i + 1].startswith("-"):
                skip_next = True
            continue
        if a.endswith(".json") and os.path.isfile(a):
            continue
        cleaned.append(a)
    return cleaned


# ------------------------------
# Configs
# ------------------------------

@dataclass
class DataConfig:
    dataset: str = "moons"              # moons|circles|blobs
    n_samples: int = 1000
    noise: float = 0.1
    centers: int = 2                    # blobs only
    cluster_std: float = 0.6            # blobs only
    val_split: float = 0.2
    test_split: float = 0.2
    seed: int = 42


@dataclass
class TrainConfig:
    run_dir: str = "runs/exp"
    hidden_sizes: List[int] = None
    lr: float = 1e-3
    weight_decay: float = 0.0
    epochs: int = 30
    batch_size: int = 32
    device: str = "auto"                # auto|cpu|gpu


# ------------------------------
# Data generation (NumPy-only)
# ------------------------------

def gen_moons(n: int, noise: float, rng: np.random.Generator) -> Tuple[np.ndarray, np.ndarray]:
    n1 = n // 2
    n2 = n - n1
    # Moon A (upper)
    t1 = rng.uniform(0, np.pi, size=n1)
    x1 = np.c_[np.cos(t1), np.sin(t1)]
    # Moon B (lower, shifted)
    t2 = rng.uniform(0, np.pi, size=n2)
    x2 = np.c_[1 - np.cos(t2), 1 - np.sin(t2) - 0.5]
    X = np.vstack([x1, x2])
    y = np.concatenate([np.zeros(n1, dtype=np.int32), np.ones(n2, dtype=np.int32)])
    X += rng.normal(scale=noise, size=X.shape)
    return X.astype(np.float32), y


def gen_circles(n: int, noise: float, rng: np.random.Generator) -> Tuple[np.ndarray, np.ndarray]:
    n1 = n // 2
    n2 = n - n1
    t1 = rng.uniform(0, 2 * np.pi, size=n1)
    t2 = rng.uniform(0, 2 * np.pi, size=n2)
    r1 = 1.0 + rng.normal(scale=noise, size=n1)
    r2 = 0.5 + rng.normal(scale=noise, size=n2)
    x1 = np.c_[r1 * np.cos(t1), r1 * np.sin(t1)]
    x2 = np.c_[r2 * np.cos(t2), r2 * np.sin(t2)]
    X = np.vstack([x1, x2]).astype(np.float32)
    y = np.concatenate([np.zeros(n1, dtype=np.int32), np.ones(n2, dtype=np.int32)])
    return X, y


def gen_blobs(n: int, centers: int, cluster_std: float, rng: np.random.Generator) -> Tuple[np.ndarray, np.ndarray]:
    # Place centers in a square and generate Gaussian blobs
    c_xy = rng.uniform(-3, 3, size=(centers, 2))
    # Distribute samples per center
    counts = [n // centers] * centers
    for i in range(n % centers):
        counts[i] += 1
    parts = []
    labels = []
    for k, cnt in enumerate(counts):
        pts = c_xy[k] + rng.normal(scale=cluster_std, size=(cnt, 2))
        parts.append(pts)
        labels.append(np.full(cnt, k, dtype=np.int32))
    X = np.vstack(parts).astype(np.float32)
    y = np.concatenate(labels)
    return X, y


def make_dataset(cfg: DataConfig) -> Tuple[np.ndarray, np.ndarray]:
    rng = np.random.default_rng(cfg.seed)
    if cfg.dataset == "moons":
        return gen_moons(cfg.n_samples, cfg.noise, rng)
    elif cfg.dataset == "circles":
        return gen_circles(cfg.n_samples, cfg.noise, rng)
    elif cfg.dataset == "blobs":
        return gen_blobs(cfg.n_samples, cfg.centers, cfg.cluster_std, rng)
    else:
        raise ValueError(f"Unknown dataset: {cfg.dataset}")


def train_val_test_split(
    X: np.ndarray, y: np.ndarray, val_split: float, test_split: float, seed: int
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    rng = np.random.default_rng(seed)
    n = len(X)
    idx = np.arange(n)
    rng.shuffle(idx)
    X = X[idx]
    y = y[idx]
    n_test = int(n * test_split)
    n_val = int(n * val_split)
    X_test, y_test = X[:n_test], y[:n_test]
    X_val, y_val = X[n_test:n_test + n_val], y[n_test:n_test + n_val]
    X_train, y_train = X[n_test + n_val:], y[n_test + n_val:]
    return X_train, y_train, X_val, y_val, X_test, y_test


# ------------------------------
# Model builders
# ------------------------------

def build_dense_model(
    input_dim: int,
    hidden_sizes: List[int],
    num_classes: int,
    weight_decay: float = 0.0,
    lr: float = 1e-3,
) -> tf.keras.Model:
    reg = tf.keras.regularizers.l2(weight_decay) if weight_decay > 0 else None
    he = tf.keras.initializers.HeNormal()

    inputs = tf.keras.Input(shape=(input_dim,), name="features")
    norm = tf.keras.layers.Normalization(name="norm")
    x = norm(inputs)
    for i, h in enumerate(hidden_sizes or []):
        x = tf.keras.layers.Dense(h, activation="relu", kernel_initializer=he, kernel_regularizer=reg, name=f"dense_{i+1}")(x)
        x = tf.keras.layers.Dropout(0.1, name=f"drop_{i+1}")(x)
    outputs = tf.keras.layers.Dense(num_classes, activation="softmax", name="logits")(x)

    model = tf.keras.Model(inputs, outputs, name="tabular_classifier")
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model


# ------------------------------
# Train / Eval / Predict / Export / Benchmark
# ------------------------------

def save_run_artifacts(run_dir: str, data_cfg: DataConfig, train_cfg: TrainConfig) -> None:
    ensure_dir(run_dir)
    save_json(os.path.join(run_dir, "data_config.json"), asdict(data_cfg))
    save_json(os.path.join(run_dir, "train_config.json"), {
        **asdict(train_cfg),
        "hidden_sizes": train_cfg.hidden_sizes or []
    })


def train_model(data_cfg: DataConfig, train_cfg: TrainConfig) -> Dict[str, Any]:
    log("Preparing data...")
    X, y = make_dataset(data_cfg)
    num_classes = int(y.max()) + 1
    X_tr, y_tr, X_val, y_val, X_te, y_te = train_val_test_split(
        X, y, data_cfg.val_split, data_cfg.test_split, seed=data_cfg.seed + 1
    )

    run_dir = train_cfg.run_dir
    ensure_dir(run_dir)
    save_run_artifacts(run_dir, data_cfg, train_cfg)

    # Build model
    log("Building model...")
    model = build_dense_model(
        input_dim=X.shape[1],
        hidden_sizes=train_cfg.hidden_sizes or [64, 64],
        num_classes=num_classes,
        weight_decay=train_cfg.weight_decay,
        lr=train_cfg.lr,
    )

    # Adapt normalization on training set
    norm_layer = model.get_layer("norm")
    norm_layer.adapt(X_tr)

    ckpt_path = os.path.join(run_dir, "best.keras")
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(ckpt_path, monitor="val_loss", save_best_only=True, verbose=1),
        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1),
        tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
        tf.keras.callbacks.CSVLogger(os.path.join(run_dir, "history.csv")),
    ]

    log("Starting training...")
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=train_cfg.epochs,
        batch_size=train_cfg.batch_size,
        verbose=1,
        callbacks=callbacks,
    )

    log("Evaluating on test set...")
    test_loss, test_acc = model.evaluate(X_te, y_te, verbose=0)
    metrics = {
        "test_loss": float(test_loss),
        "test_acc": float(test_acc),
        "n_train": int(len(X_tr)),
        "n_val": int(len(X_val)),
        "n_test": int(len(X_te)),
    }
    save_json(os.path.join(run_dir, "metrics.json"), metrics)

    # Save a final copy of the model (best already saved via checkpoint)
    model.save(os.path.join(run_dir, "latest.keras"))
    log(f"Done. Run dir: {run_dir}")
    return {"run_dir": run_dir, "metrics": metrics, "history_keys": list(history.history.keys())}


def reconstruct_configs(run_dir: str) -> Tuple[DataConfig, TrainConfig]:
    data_cfg = DataConfig(**load_json(os.path.join(run_dir, "data_config.json")))
    tcfg_dict = load_json(os.path.join(run_dir, "train_config.json"))
    # Ensure list type for hidden sizes
    tcfg_dict["hidden_sizes"] = list(tcfg_dict.get("hidden_sizes", []))
    train_cfg = TrainConfig(**tcfg_dict)
    return data_cfg, train_cfg


def evaluate_model(run_dir: str) -> Dict[str, Any]:
    log(f"Loading configs from: {run_dir}")
    data_cfg, _ = reconstruct_configs(run_dir)

    # Re-generate deterministic dataset and splits
    X, y = make_dataset(data_cfg)
    X_tr, y_tr, X_val, y_val, X_te, y_te = train_val_test_split(
        X, y, data_cfg.val_split, data_cfg.test_split, seed=data_cfg.seed + 1
    )

    # Load best checkpoint
    best_path = os.path.join(run_dir, "best.keras")
    if not os.path.exists(best_path):
        raise FileNotFoundError(f"Best checkpoint not found: {best_path}")
    model = tf.keras.models.load_model(best_path)

    log("Evaluating best checkpoint on test set...")
    loss, acc = model.evaluate(X_te, y_te, verbose=0)
    out = {"test_loss": float(loss), "test_acc": float(acc)}
    log(f"Test — loss: {loss:.4f}, acc: {acc:.4f}")
    return out


def predict_csv(run_dir: str, csv_path: str, output: Optional[str], features: Optional[List[str]], include_proba: bool) -> str:
    try:
        import pandas as pd
    except Exception as e:
        raise RuntimeError("pandas is required for CSV prediction. Please `pip install pandas`.") from e

    best_path = os.path.join(run_dir, "best.keras")
    if not os.path.exists(best_path):
        raise FileNotFoundError(f"Best checkpoint not found: {best_path}")
    model = tf.keras.models.load_model(best_path)

    df = pd.read_csv(csv_path)
    if features:
        missing = [c for c in features if c not in df.columns]
        if missing:
            raise ValueError(f"Missing feature columns in CSV: {missing}")
        X = df[features].to_numpy(dtype=np.float32)
    else:
        # Use all numeric columns
        num_df = df.select_dtypes(include=[np.number])
        if num_df.empty:
            raise ValueError("No numeric columns found. Provide --features.")
        X = num_df.to_numpy(dtype=np.float32)

    preds = model.predict(X, verbose=0)
    pred_labels = np.argmax(preds, axis=1)

    out_df = df.copy()
    out_df["prediction"] = pred_labels
    if include_proba:
        for i in range(preds.shape[1]):
            out_df[f"proba_{i}"] = preds[:, i]

    if output is None or output == "":
        base, ext = os.path.splitext(csv_path)
        output = base + ".predictions.csv"
    out_df.to_csv(output, index=False)
    log(f"Wrote predictions to: {output}")
    return output


def export_onnx(run_dir: str, output: Optional[str]) -> str:
    best_path = os.path.join(run_dir, "best.keras")
    if not os.path.exists(best_path):
        raise FileNotFoundError(f"Best checkpoint not found: {best_path}")
    model = tf.keras.models.load_model(best_path)

    if output is None or output == "":
        output = os.path.join(run_dir, "model.onnx")

    try:
        import tf2onnx
        import onnx  # noqa: F401 (ensures ONNX is available)
    except Exception as e:
        raise RuntimeError("ONNX export requires tf2onnx and onnx. Try: pip install tf2onnx onnx") from e

    log("Converting to ONNX...")
    spec = (tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype, name=model.inputs[0].name),)
    model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, opset=13)
    with open(output, "wb") as f:
        f.write(model_proto.SerializeToString())
    log(f"Exported ONNX to: {output}")
    return output


def benchmark(data_cfg: DataConfig, train_cfg: TrainConfig, seeds: List[int]) -> Dict[str, Any]:
    results = []
    log(f"Benchmarking seeds: {seeds}")
    for s in seeds:
        run_tag = f"{train_cfg.run_dir}_seed{s}"
        tcfg = TrainConfig(
            run_dir=run_tag,
            hidden_sizes=train_cfg.hidden_sizes,
            lr=train_cfg.lr,
            weight_decay=train_cfg.weight_decay,
            epochs=train_cfg.epochs,
            batch_size=train_cfg.batch_size,
            device=train_cfg.device,
        )
        dcfg = DataConfig(
            dataset=data_cfg.dataset,
            n_samples=data_cfg.n_samples,
            noise=data_cfg.noise,
            centers=data_cfg.centers,
            cluster_std=data_cfg.cluster_std,
            val_split=data_cfg.val_split,
            test_split=data_cfg.test_split,
            seed=s,
        )
        out = train_model(dcfg, tcfg)
        results.append({"seed": s, **out["metrics"]})

    # Aggregate
    test_accs = [r["test_acc"] for r in results]
    test_losses = [r["test_loss"] for r in results]
    summary = {
        "seeds": seeds,
        "mean_acc": float(np.mean(test_accs)),
        "std_acc": float(np.std(test_accs)),
        "mean_loss": float(np.mean(test_losses)),
        "std_loss": float(np.std(test_losses)),
        "n_runs": len(seeds),
    }

    ensure_dir(train_cfg.run_dir)
    save_json(os.path.join(train_cfg.run_dir, "benchmark_summary.json"), summary)
    # Save per-seed CSV
    try:
        import pandas as pd
        import csv
        pd.DataFrame(results).to_csv(os.path.join(train_cfg.run_dir, "benchmark_results.csv"), index=False)
    except Exception:
        # Minimal CSV writer fallback
        p = os.path.join(train_cfg.run_dir, "benchmark_results.csv")
        keys = list(results[0].keys())
        with open(p, "w", newline="", encoding="utf-8") as f:
            import csv as _csv
            w = _csv.DictWriter(f, fieldnames=keys)
            w.writeheader()
            w.writerows(results)
    log(f"Benchmark summary: acc={summary['mean_acc']:.4f}±{summary['std_acc']:.4f}, "
        f"loss={summary['mean_loss']:.4f}±{summary['std_loss']:.4f}")
    return {"results": results, "summary": summary}


# ------------------------------
# Command handlers (CLI)
# ------------------------------

def cmd_train(args: argparse.Namespace) -> None:
    data_cfg = DataConfig(
        dataset=args.dataset,
        n_samples=args.nsamples,
        noise=args.noise,
        centers=args.centers,
        cluster_std=args.clusterstd,
        val_split=args.valsplit,
        test_split=args.testsplit,
        seed=args.seed,
    )
    hidden = [int(x) for x in args.hidden_sizes.split(",") if x.strip()]
    train_cfg = TrainConfig(
        run_dir=args.rundir,
        hidden_sizes=hidden,
        lr=args.lr,
        weight_decay=args.weightdecay,
        epochs=args.epochs,
        batch_size=args.batchsize,
        device=args.device,
    )
    log("Starting training...")
    out = train_model(data_cfg, train_cfg)
    log(f"Done. Run dir: {out['run_dir']}")


def cmd_eval(args: argparse.Namespace) -> None:
    log("Evaluating best checkpoint...")
    evaluate_model(args.rundir)


def cmd_predict(args: argparse.Namespace) -> None:
    feats = [c.strip() for c in args.features.split(",") if c.strip()] if args.features else None
    predict_csv(args.rundir, args.csv, output=args.output, features=feats, include_proba=(not args.noproba))


def cmd_export_onnx(args: argparse.Namespace) -> None:
    export_onnx(args.rundir, args.output)


def cmd_benchmark(args: argparse.Namespace) -> None:
    data_cfg = DataConfig(
        dataset=args.dataset,
        n_samples=args.nsamples,
        noise=args.noise,
        centers=args.centers,
        cluster_std=args.clusterstd,
        val_split=args.valsplit,
        test_split=args.testsplit,
        seed=0,  # overridden per run
    )
    hidden = [int(x) for x in args.hidden_sizes.split(",") if x.strip()]
    train_cfg = TrainConfig(
        run_dir=args.rundir,
        hidden_sizes=hidden,
        lr=args.lr,
        weight_decay=args.weightdecay,
        epochs=args.epochs,
        batch_size=args.batchsize,
        device=args.device,
    )
    seeds = [int(s) for s in args.seeds.split(",") if s.strip()]
    benchmark(data_cfg, train_cfg, seeds)


# ------------------------------
# CLI parser
# ------------------------------

def build_parser() -> argparse.ArgumentParser:
    p = argparse.ArgumentParser(
        prog="metaintel",
        description="MetaIntelligence CLI: train/eval/predict/benchmark/export-onnx",
        formatter_class=argparse.ArgumentDefaultsHelpFormatter,
    )
    sub = p.add_subparsers(dest="command", required=True)

    # Train
    t = sub.add_parser("train", help="Train a model", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    t.add_argument("--dataset", type=str, default="moons", choices=["moons", "circles", "blobs"], help="Synthetic dataset")
    t.add_argument("--nsamples", type=int, default=1000, help="Total samples")
    t.add_argument("--noise", type=float, default=0.1, help="Noise level")
    t.add_argument("--centers", type=int, default=2, help="For blobs: number of centers")
    t.add_argument("--clusterstd", type=float, default=0.6, help="For blobs: std of clusters")
    t.add_argument("--valsplit", type=float, default=0.2, help="Validation split")
    t.add_argument("--testsplit", type=float, default=0.2, help="Test split")
    t.add_argument("--seed", type=int, default=42, help="Random seed")
    t.add_argument("--hidden-sizes", type=str, default="64,64", help="Comma-separated hidden sizes")
    t.add_argument("--lr", type=float, default=1e-3, help="Learning rate")
    t.add_argument("--weightdecay", type=float, default=0.0, help="L2 weight decay (regularizer)")
    t.add_argument("--epochs", type=int, default=30, help="Training epochs")
    t.add_argument("--batchsize", type=int, default=32, help="Batch size")
    t.add_argument("--device", type=str, default="auto", choices=["auto", "cpu", "gpu"], help="Device preference (informational)")
    t.add_argument("--rundir", type=str, required=True, help="Run directory")
    t.set_defaults(func=cmd_train)

    # Eval
    e = sub.add_parser("eval", help="Evaluate best checkpoint on test set", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    e.add_argument("--rundir", type=str, required=True, help="Run directory")
    e.set_defaults(func=cmd_eval)

    # Predict
    pr = sub.add_parser("predict", help="Predict on CSV (tabular only)", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    pr.add_argument("--rundir", type=str, required=True, help="Run directory with trained model")
    pr.add_argument("--csv", type=str, required=True, help="Input CSV path")
    pr.add_argument("--output", type=str, default="", help="Output CSV path")
    pr.add_argument("--features", type=str, default="", help="Comma-separated feature columns (default: all numeric)")
    pr.add_argument("--noproba", action="store_true", help="Do not include class probabilities")
    pr.set_defaults(func=cmd_predict)

    # Export ONNX
    ex = sub.add_parser("export-onnx", help="Export best checkpoint to ONNX", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    ex.add_argument("--rundir", type=str, required=True, help="Run directory")
    ex.add_argument("--output", type=str, default="", help="ONNX output path")
    ex.set_defaults(func=cmd_export_onnx)

    # Benchmark
    b = sub.add_parser("benchmark", help="Benchmark multiple seeds", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    b.add_argument("--dataset", type=str, default="moons", choices=["moons", "circles", "blobs"], help="Synthetic dataset")
    b.add_argument("--nsamples", type=int, default=1000, help="Total samples")
    b.add_argument("--noise", type=float, default=0.1, help="Noise level")
    b.add_argument("--centers", type=int, default=2, help="For blobs: number of centers")
    b.add_argument("--clusterstd", type=float, default=0.6, help="For blobs: std of clusters")
    b.add_argument("--valsplit", type=float, default=0.2, help="Validation split")
    b.add_argument("--testsplit", type=float, default=0.2, help="Test split")
    b.add_argument("--hidden-sizes", type=str, default="64,64", help="Comma-separated hidden sizes")
    b.add_argument("--lr", type=float, default=1e-3, help="Learning rate")
    b.add_argument("--weightdecay", type=float, default=0.0, help="L2 weight decay (regularizer)")
    b.add_argument("--epochs", type=int, default=20, help="Training epochs per seed")
    b.add_argument("--batchsize", type=int, default=32, help="Batch size")
    b.add_argument("--device", type=str, default="auto", choices=["auto", "cpu", "gpu"], help="Device preference (informational)")
    b.add_argument("--rundir", type=str, required=True, help="Root run directory for benchmark summary")
    b.add_argument("--seeds", type=str, default="1,2,3,4,5", help="Comma-separated seeds")
    b.set_defaults(func=cmd_benchmark)

    return p


# ------------------------------
# Main (Notebook/Colab-safe)
# ------------------------------

def main(argv: Optional[List[str]] = None):
    import sys
    raw_args = argv if argv is not None else sys.argv[1:]
    filtered_args = sanitize_argv(raw_args)

    parser = build_parser()

    if not filtered_args:
        parser.print_help()
        print("\n📌 Example usage (in notebooks):")
        print('main(["train", "--dataset", "moons", "--epochs", "30", "--rundir", "runs/moonstest"])')
        print('main(["eval", "--rundir", "runs/moonstest"])')
        print('main(["export-onnx", "--rundir", "runs/moonstest"])')
        return

    try:
        args = parser.parse_args(filtered_args)
    except SystemExit:
        parser.print_help()
        print("\n📌 Example usage (in notebooks):")
        print('main(["train", "--dataset", "moons", "--epochs", "30", "--rundir", "runs/moonstest"])')
        print('main(["eval", "--rundir", "runs/moonstest"])')
        print('main(["export-onnx", "--rundir", "runs/moonstest"])')
        return

    func = getattr(args, "func", None)
    if func is None:
        parser.print_help()
        print("\n📌 Example usage (in notebooks):")
        print('main(["train", "--dataset", "moons", "--epochs", "30", "--rundir", "runs/moonstest"])')
        print('main(["eval", "--rundir", "runs/moonstest"])')
        print('main(["export-onnx", "--rundir", "runs/moonstest"])')
        return

    func(args)


if __name__ == "__main__":
    main()